In [1]:
#imports
import os
import matplotlib.pyplot as plt
import torch
import numpy as np
import fitz  # pymupdf
import pandas as pd
import re
import spacy
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM


In [2]:
nlp = spacy.load("de_core_news_sm")
# Initialize the embedding model and move to the correct device
embedding_model = SentenceTransformer("all-mpnet-base-v2")
torch.cuda.empty_cache()


In [3]:
# Helper function to clean up text
def text_formatter(text: str) -> str:
    return text.replace("\n", " ").strip()

# PDF reading function
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in enumerate(doc):
        text = text_formatter(page.get_text())
        pages_and_texts.append({
            "page_number": page_number,
            "text": text,
            "sentences": [str(sentence) for sentence in nlp(text).sents]
        })
    return pages_and_texts
pdf_path="Sicherung-des-Schienenverkehrs.pdf"
pages_and_texts = open_and_read_pdf(pdf_path)
pages_and_texts

[{'page_number': 0,
  'text': 'Sicherung des  Schienenverkehrs Ulrich Maschek Grundlagen und Planung der Leit- und  Sicherungstechnik 5. Auflage',
  'sentences': ['Sicherung des  Schienenverkehrs Ulrich Maschek Grundlagen und Planung der Leit- und  Sicherungstechnik 5. Auflage']},
 {'page_number': 1,
  'text': 'Sicherung des Schienenverkehrs',
  'sentences': ['Sicherung des Schienenverkehrs']},
 {'page_number': 2,
  'text': 'Ulrich Maschek Sicherung des Schienenverkehrs Grundlagen und Planung der Leit- und Sicherungstechnik 5.,aktualisierte und erweiterte Auflage',
  'sentences': ['Ulrich Maschek',
   'Sicherung des Schienenverkehrs Grundlagen und Planung der Leit- und Sicherungstechnik',
   '5.,aktualisierte und erweiterte Auflage']},
 {'page_number': 3,
  'text': 'Ulrich Maschek Technische Universität Dresden Dresden, Deutschland ISBN 978-3-658-37632-1 ISBN 978-3-658-37633-8 (eBook) https://doi.org/10.1007/978-3-658-37633-8 Die Deutsche Nationalbibliothek verzeichnet diese Publikatio

In [7]:
# Splitting function for sentence chunks
def split_list(input_list: list, slice_size: int) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Chunk preparation with embeddings
def prepare_chunks(pages_and_texts: list[dict], chunk_size=10):
    pages_and_chunks = []
    for item in pages_and_texts:
        sentence_chunks = split_list(item["sentences"], chunk_size)
        for chunk in sentence_chunks:
            joined_chunk = " ".join(chunk).strip()
            pages_and_chunks.append({
                "page_number": item["page_number"],
                "sentence_chunk": joined_chunk,
                "embedding": embedding_model.encode(joined_chunk, convert_to_tensor=True)
            })
    return pages_and_chunks


In [8]:
# Main pipeline initialization
def initialize_pipeline(pdf_path: str):
    print("Loading PDF and processing text...")
    pages_and_texts = open_and_read_pdf(pdf_path)
    print("Preparing sentence chunks and embeddings...")
    pages_and_chunks = prepare_chunks(pages_and_texts)
    return pages_and_chunks

In [9]:
pages_and_texts = initialize_pipeline(pdf_path="Sicherung-des-Schienenverkehrs.pdf")

Loading PDF and processing text...
Preparing sentence chunks and embeddings...
